HOW TO RUN

roscore

open graphical tools

roslaunch assignment_2 map.launch

rosrun assignment_2 ekf_node.py

cd dataset_ws

rosbag play assignment_2.bag

![](Picture.jpg)

# Assignment 2: Recursive Bayes Filters and Robot Localisation

## Background

Single sensor has limited capabilities. In the context of robot localisation, for example, robot odometry drifts over time. Although GPS provides global positions, it can only achieve moderate accuracy locally and suffers from noises due to building occlusion, dense trees etc. Moreover, normal GPS only has about several meters accuracy, which is not enough for many robotic applications. Therefore, recursive Bayes filter based multi-sensor fusion is often used to fuse odometry with GPS for outdoor mobile robotics, reducing the drifts of odometry and smoothing GPS locations. 

## Objectives

This Assignment focuses on developing an Extended Kalman filter (EKF) or Particle filter (PF) algorithm to fuse robot visual odometry and GPS for accurate robot localisation. It aims to consolidate the lectures by practicing and understanding how to design and tune a filter to solve a real-world problem. Since this Assignment is provided with some real robot data, the algorithms developed can be easily tested and deployed on real robots. 

## Dataset and Sensors

### ROSBag Dataset

We provide a ROSBag dataset generated from the [Oxford RobotCar Dataset](https://robotcar-dataset.robots.ox.ac.uk/). The dataset which includes the following ROS topics

- Visual odometry topic <code>/visual_odometry</code>: It is a ROS <code>nav_msgs/Odometry</code> message which provides the the 3D transformation from the source image to destination image in the twist. Note this is the 6 Degree-of-Freedom relative transformation in the camera local frame (defined as <mark>x-forward, y-right, z-down</mark>) instead of velocity. Since this assignment is a mobile robot localisation in 2D, you only need to use the x, y and yaw motion.
- GPS topic <code>/gps</code>: This is given as ROS <code>sensor_msgs/NavSatFix</code> type. The GPS data is composed of latitude (deg), longitude (deg), altitude (m) and uncertainty (m) at about 5Hz. You could use the ROS [geonav_transform package](https://wiki.ros.org/geonav_transform) to convert latitude and longitude into XYZ coordinates. It is sometimes noise as seen in the figure below. We also provide the necessary python files in the rosject to enable you to do the conversions.
![GPS on GoogleMap](gps.png)
- Camera image topic <code>/image_centre</code>: It is a ROS <code>sensor_msgs/Image</code> message provided for visualisation purpose. It is not required to work on this assignment.
<img src="camera.gif" alt="Camera video" width="320"/>

The ROSBag <code>assignment2.bag</code> is in the <code>datasets_ws</code> folder of this project. You can use the <code>rosbag play</code> command to play the data for your assignment. The <code>rosbag play assignment2.bag> </code> will start the replay of the bag. Therefore you should start your filtering node <b>before</b> you start playing the rosbag

### Sensor Transformation

The sensor layout and transformations can be found from the [documentation](https://robotcar-dataset.robots.ox.ac.uk/documentation/) of RobotCar Dataset. You may want to use the transformation between the camera and GPS for fusion. For simplicity, it is also ok to assume the camera (visual odometry) and GPS are located at the same position for this assignment.

## Tasks (30 Points Total)

Assignment 2 is to fuse visual odometry with GPS using the <code>assignment2.bag</code> ROSBag data provided. You must use an EKF to do so. Before starting the following tasks, please:
- Use the ros package named <code>assignment2></code> in the catkin_ws workspace. This should be the package which contains **ALL your codes for this Assignment 2**.
- This package already has a file named ekf_node.py
- Use this file to write the new code. The placeholder for the code is already available 


### Task 1 Motion Model (2 points)
The motion model is provided to you in the call back for the visual_odometry message. Using the visual odometry motion model, we are then generating a dead-reckoning (compounded motions from visual odometry) using the visual odometry data, and we publish a new odometry topic named <code>odometry_vo></code> as <code>nav_msgs/Odometry</code> type in the /odom frame. It has its <code>pose</code> populated as an accumulated odometry trajectory. We can visualise the accumulated odometry in red in rviz.
You will need to launch rviz and add the message to be able to see it.

- Visualise the accumulated odometry trajectory in red colour in rviz and save a screenshot named <code>screenshot_motion_&lt;your_name>.jpg</code>. (2 point) 

### Task 2: Measurement Model (2 points)
We have subsribed to the measurement model for you and used the correct conversions to transfer data from gps coordinates to UTM (x,y). Note that the gps data provides and instantaneous estimate of the gps measurement noise (Rt) which can be used to make the filter adapt to the changes in measurement noise. You can also use a fixed Rt if you want. We then generate a new odometry message odometry_gps
- Visualise the gps trajectory in red colour in rviz and save a screenshot named <code>screenshot_motion_&lt;your_name>.jpg</code>. (2 point) 

### Task 3: Filter Algorithm (20 Points)

You will need to design your own EKF or PF algorithm (feel free to choose the one you prefer) to achieve a good localisation by fusing the odometry with GPS. The algorithm only needs to perform 2D localisation, i.e., estimating (x, y, yaw).
- Programme the prediction step with predictions on both the state and covariance. (10 points)
- Programme the update step correcting both the state and covariance. (10 points)
- Plot the rviz<code>fused_pose_&lt;your_name></code> in green colour in rviz and save a screenshot named <code>screenshot_fusion_&lt;your_name>.jpg</code>. (1 point)

### Task 4: Visualisation and Comparison (3 Points)

- Visualise your fused pose in rviz using a [Satellite plugin](https://github.com/gareth-cross/rviz_satellite). This plugin allows you to position the robot in the corresponding real place on Earth. (1 point)
- Visualise the results against your ones in Task 3 in rviz and save a screenshot named <code>screenshot_comp_&lt;your_name>.jpg</code>. (2 point)

Note that we have also included a particle filter version that you can use to compare your results to:
To run it:
- roscore
- rosrun assignment2 topics_node.py
- rosrun assignment2 pf_node.py
- rosbag play datasets_ws/assignment2.bag
- rosrun rviz rviz

### Task 5: Evaluation (3 Points)


The localisation results will be evaluated by comparing with the ground truth. I have now setup a system for you to test your results before submission. In the assignment_2 package, you will find an evaluation directory containing the following files:
evaluation.py and pose_slam.txt. To run the evaluatio, do the following:

- copy your fused_pose_exp.csv file into the evaluation directory
- run the evaluation.py file as: 'python evaluation.py'.This will draw graphs comparing your estimate and the ground truth and give you an estimate of your MSE. 

- Please email me a csv file (more details in [Submission](#Submission)) which includes your fused 2D pose estimates in a local coordinate, i.e., the first pose is [0,0,0] instead of global GPS frame. It should have 3020 lines in total, including the first title row, in the following format. An template named fused_estimates_template.csv can also be found on Canvas.

| timestamp        | x (m)    | y (m)    | yaw (radian) |
|:---              |:---      |:---      |:---          |
| 1403772869717912 | 0.0000   | 0.0000   | 0.0000       |
| 1403772869780415 | x.xxxx   | x.xxxx   | x.xxxx       |
| 1403772869842914 | x.xxxx   | x.xxxx   | x.xxxx       |
| ......           |  ......  |  ......  | ......       |

This task will be evaluted according to the mean absolute localisation accuracy (MALA):
- 1 point: 30 meters < MALA;
- 2 points: 15 meters < MALA <= 30 meters;
- 3 points: MALA <15 meters.


## Submission

<span style="color:red">Deadline: 21th November 2021</span>

1. Once you have finished, save your ROSject as assignment2_&lt;your_name> and save your localisation results into a csv file named fused_estimates_&lt;your_name>.csv.
1. send the following files to Yvan Petillot via email (_y.r.petillot_ AT _hw.ac.uk_ )
    1. your fused_estimates_&lt;your_name>.csv file
    1. your ROSject link 
    1. the screenshots saved: screenshot_motion_&lt;your_name>.jpg, screenshot_fusion_&lt;your_name>.jpg, screenshot_comp_&lt;your_name>.jpg
1. Download your project from ROSject (using the download button) as a tar or zip file named as assignment2_&lt;your_name> and submit on Canvas via _Assessments_ -> _Assessment2_. Please remove the rosbag dataset file as it is very big.


This is an assignment created for B31YS Robotics Systems Science course at Heriot-Watt University, UK. Created by Yvan Petillot. E-mail: _y.r.petillot_ AT _hw.ac.uk_